In [12]:
import torch
import datetime
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision.models import densenet
from torchvision.datasets import coco
from torchvision import transforms

In [17]:
BASE = '/Users/peterspradling/CS349D/'
m = densenet.densenet121(pretrained=True)

/Users/peterspradling/anaconda3/envs/torch/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


In [18]:
ANN_PATH = 'coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = 'coco/cocoapi/images/val2017/'
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
def merge_coco(l):
    return torch.stack([x[0] for x in l])
c_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=test_transform, )
eval_loader = torch.utils.data.DataLoader(c_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.92s)
creating index...
index created!


In [27]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

def densenet_measure_forward(self, x):
    print('Image size:', compute_mem_size(x))
    features = self.features(x)
    
    for module in self.features:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
        
    
    t2 = datetime.datetime.now()
    out = F.relu(features, inplace=True)
    t3 = datetime.datetime.now()
    dt2 = t3 - t2
    print('After relu (' + str(dt1.total_seconds()) + '):', compute_mem_size(out))
    
    t4 = datetime.datetime.now()
    out = F.avg_pool2d(out, kernel_size=7, stride=1).view(features.size(0), -1)
    t5 = datetime.datetime.now()
    dt3 = t5 - t4
    print('After avg_pool2d (' + str(dt2.total_seconds()) + '):', compute_mem_size(out))
    
    #for module in self.classifier:
    t6 = datetime.datetime.now()
    #x = module(x)
    x = self.classifier(out)
    t7 = datetime.datetime.now()
    dt4 = t7 - t6
    canon_name = type(self.classifier).__name__
    print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
    
    return out

In [28]:
m.eval()
for batch in eval_loader:
    densenet_measure_forward(m, batch)
    break

Image size: 602112.0
After Conv2d(0.009525):  3211264.0
After BatchNorm2d(0.002462):  3211264.0
After ReLU(0.000591):  3211264.0
After MaxPool2d(0.008842):  802816.0
After _DenseBlock(0.120857):  3211264.0
After _Transition(0.011219):  401408.0
After _DenseBlock(0.091882):  1605632.0
After _Transition(0.009451):  200704.0
After _DenseBlock(0.131066):  802816.0
After _Transition(0.007704):  100352.0
After _DenseBlock(0.054442):  200704.0
After BatchNorm2d(0.002762):  200704.0
After relu (0.002762): 200704.0
After avg_pool2d (0.0003): 4096.0
After Linear(0.002762):  4000.0


In [11]:

#transforms.ToPILImage()(sample_input)